In [1]:
import sys
from PIL import Image
import os
import json
from tqdm import tqdm

def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.rindex( last, start )
        return s[start:end]
    except ValueError:
        return ""

path = "original/"
tasks = {}
for filea in os.listdir(path):
    if filea.endswith(".png"):
        fileb = os.path.join(path, filea)
        name = find_between(filea,'HIT_','_')
        value = []
        if tasks.has_key(name):
            value = tasks[name]
            value.append(fileb)
        else:
            value = [fileb]
        
        tasks[name] = value

        
for name, value in tqdm(tasks.iteritems()):
    images_list = []
    for i in range(len(value)-1):
        images_list.append(path+"HIT_"+name+"_"+str(i+1)+".png")
    
    images = map(Image.open, images_list)
    widths, heights = zip(*(i.size for i in images))

    total_width = sum(widths)
    max_height = max(heights)

    new_im = Image.new('RGB', (total_width, max_height))

    x_offset = 0
    for im in images:
        new_im.paste(im, (x_offset,0))
        x_offset += im.size[0]

    new_im.save(name+'.jpg')
    
    # now save final
    images_list = []
    images_list.append(path+"HIT_"+name+"_"+str(len(value)-1)+".png")
    
    images = map(Image.open, images_list)
    widths, heights = zip(*(i.size for i in images))

    total_width = sum(widths)
    max_height = max(heights)

    new_im = Image.new('RGB', (total_width, max_height))

    x_offset = 0
    for im in images:
        new_im.paste(im, (x_offset,0))
        x_offset += im.size[0]

    new_im.save(name+'_final.jpg')

692it [03:27,  3.22it/s]


In [2]:
html = "<!DOCTYPE html>\n<html>\n<body>\n<h2>The End</h2>\n</body>\n</html>\n"
number = str(692)

count = 0
for name in tqdm(sorted(tasks.iterkeys())):
    count = count + 1
    value = tasks[name]
    instructions = ""
    with open(name+'_ins.json') as json_file:  
        data = json.load(json_file)
        for pp in data['instructions']:
            instructions += "<p>"+pp+"</p>\n"
        
    html = html.replace('<h2>The End</h2>\n',
                        ("\n"
    "<h2>Count: "+str(count)+"&nbsp;&nbsp;&nbsp; Index: "+name.replace("_configs_", "&nbsp;&nbsp;&nbsp; <font color='red'><strong>Config: ")
                         +"&nbsp;&nbsp;&nbsp; Instructions: "+data['ins_name']+"</strong></font></h2>\n"
    "<img src='images_data_collage_"+number+"/"+name+"_final.jpg'  style='float:right; width: 400px'/>\n"
    "<div>\n"+instructions+"</div>\n"
    "<img src='images_data_collage_"+number+"/"+name+".jpg' style='display:block; width:100%; height:auto;'>\n\n"
    "<h2>The End</h2>\n"
                       )).encode("ascii")

with open("table-setting reconstructed images "+number+".html", "w") as text_file:
    text_file.write("{0}".format(html))
    
# print(html)

100%|██████████| 692/692 [00:02<00:00, 301.84it/s] 


In [4]:
import boto3
import collections
import os

def upload_html_to_s3(
    html_filepath = None,
    s3_bucket_name = None,
    keyname = None):

    # Uploads an HTML file on your computer to an s3 bucket on your AWS account.
    s3 = boto3.resource('s3')
    s3.meta.client.upload_file(html_filepath, s3_bucket_name, keyname,ExtraArgs= {'ContentType': 'text/html'})

    object_acl = s3.ObjectAcl(s3_bucket_name, keyname)
    response = object_acl.put(ACL='public-read')
    url = os.path.join('https://s3.amazonaws.com', s3_bucket_name, keyname)
    print(url)

    return url

url = upload_html_to_s3(html_filepath ='table-setting reconstructed images '+number+'.html', 
                   s3_bucket_name = 'nusrls', # Change this to the name of the bucket you created.
                   keyname = 'table-setting_reconstructed_images_'+number+'.html')

https://s3.amazonaws.com/nusrls/table-setting_reconstructed_images_692.html
